In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv('Resources/lending_data.csv')
# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]

# Separate the X variable, the features
X = lending_df[['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt']]

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.sample(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
24840,8800.0,6.865,45200,0.336283,3,0,15200
54704,8800.0,6.878,45300,0.337748,3,0,15300
60294,9900.0,7.349,49800,0.397590,4,0,19800
48692,9500.0,7.178,48100,0.376299,4,0,18100
57763,10700.0,7.665,52700,0.430740,5,1,22700
12528,8900.0,6.891,45400,0.339207,3,0,15400
7431,11400.0,7.952,55400,0.458484,5,1,25400
12081,9500.0,7.148,47900,0.373695,4,0,17900
8112,10100.0,7.433,50500,0.405941,4,1,20500
63595,9900.0,7.318,49500,0.393939,4,0,19500


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
label_counts = y.value_counts()
print(label_counts)

0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver = 'lbfgs', max_iter = 200, random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
y_pred = model.predict(X_test)
df_results = pd.DataFrame({
    "y_test": y_test,
    "pred": y_pred
})

df_results.sample(10)

,y_test,pred
759,0,0
22226,0,0
23857,0,0
40482,0,0
65294,0,0
74502,0,0
9975,0,0
44922,0,0
26959,0,0
60494,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy= round(accuracy,3)
print("Accuracy:", accuracy)

Accuracy: 0.992


In [11]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mtx)

Confusion Matrix:
[[18663   102]
 [   56   563]]


In [12]:
# Print the classification report for the model
from sklearn.metrics import classification_report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  In contrast to the initial dataset, the quantity of sound loans surpasses that of problematic loans. The model exhibits a commendable overall accuracy of 99%, with a precision score of 100% for class 0 (representing healthy loans) and a respectable 85% precision for class 1 (indicating unhealthy loans). Furthermore, the recall scores are notably high, registering 99% for predictions associated with class 0 and 91% for identifying high-risk loans labeled as class 1.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
r_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = r_model.fit_resample(X_train, y_train)

In [19]:
# Count the distinct values of the resampled labels data

label_counts = y_resampled.value_counts()
num_distinct_labels = len(label_counts)

print("Number of distinct labels:", num_distinct_labels)

print(label_counts)

Number of distinct labels: 2
0    56271
1    56271
Name: loan_status, dtype: int64


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [20]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(solver = 'lbfgs', max_iter = 200, random_state=1)
# Fit the model using training data
lr_model.fit(X_train, y_train)

# Fit the model using the resampled training data
lr_model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [21]:
# Print the balanced_accuracy score of the model 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, testing_predictions)
accuracy = round(accuracy, 3)
print("Accuracy:", accuracy)

Accuracy: 0.994


In [22]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(y_test, testing_predictions)
print("Confusion Matrix:")
print(confusion_mtx)

Confusion Matrix:
[[18649   116]
 [    4   615]]


In [23]:
# Print the classification report for the model
from sklearn.metrics import classification_report
classification_rep = classification_report(y_test, testing_predictions)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The accuracy score for both predictive models stands impressively high at 99%. Upon scrutinizing the confusion matrix, it becomes evident that the model trained on oversampled data outperformed in predicting false negatives, with only 4 loans of type 0 being misclassified. Much like the prior model, precision for type 0 loans remained at 100%, and for type 1 loans, it was 84%. Notably, the recall score for unhealthy loans saw improvement compared to the earlier model. In summary, while the original model demonstrated commendable accuracy, the oversampled data model appears slightly superior in its ability to make more precise predictions.